# Deploying agent to reasoning engine

This notebook covers how we can restructure the agent code and ultimately deploy the agent to google vertex ai.

## Required Software Installed Locally

* Python version 3.9, 3.10, or 3.11. **Python3.12 will not work**.

* If you are using VSCode, please install Jupyter Notebook extensions.

* Jupyter notebook. Please follow this [installation guide](https://docs.jupyter.org/en/stable/install.html). You may choose whether you want to install classic jupyter notebook or jupyterlab (the next-gen web ui for jupyter)

    * [Classic jupyter notebook installation guide](https://docs.jupyter.org/en/stable/install/notebook-classic.html)

    * [Jupyterlab installation guide](https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html)

* Google Cloud CLI. Please follow this [installation guide](https://cloud.google.com/sdk/docs/install-sdk)

### Installing dependencies

In [ ]:
%%writefile requirements.txt

google-cloud-aiplatform
google-cloud-aiplatform[langchain]
google-cloud-aiplatform[reasoningengine]
langchain
langchain_core
langchain_community
langchain-google-vertexai==2.0.8
cloudpickle
pydantic==2.9.2
langchain-google-community
google-cloud-discoveryengine
nest-asyncio
asyncio==3.4.3
asyncpg==0.29.0
cloud-sql-python-connector[asyncpg]
langchain-google-cloud-sql-pg
numpy
pandas
pgvector
psycopg2-binary
langchain-openai
langgraph
traceloop-sdk
opentelemetry-instrumentation-google-generativeai
opentelemetry-instrumentation-langchain
opentelemetry-instrumentation-vertexai
python-dotenv

In [ ]:
!pip install --upgrade -r requirements.txt

## Setting up Google Cloud Account

#### Recommended account setup

if you are running this in jupyter notebook locally, you may need to login to google cloud by running the following command from terminal:

```
gcloud auth login
gcloud auth application-default login
```

If you are using Google Colabs, you need to authenticate with your google account by running the following notebook cell. 

> Please remember that you will need to do this on each jupyter notebook during this workshop

In [ ]:
# #@markdown ###Authenticate your Google Cloud Account and enable APIs.
# # Authenticate gcloud.
from google.colab import auth
auth.authenticate_user()

In [ ]:
import requests
from typing import Optional, List
from IPython.display import display, Markdown

from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.tools.retriever import create_retriever_tool
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import StructuredTool
from langchain.memory import ChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_google_cloud_sql_pg import PostgresChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)

from langchain_google_cloud_sql_pg import (
    PostgresEngine,
    PostgresVectorStore,
)

from vertexai.preview import reasoning_engines 
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_vertexai import HarmBlockThreshold, HarmCategory

In [ ]:
project_id = "imrenagi-devfest-2024"  # @param {type:"string"}
region = "us-central1" #change this to project location
staging_bucket_name = "devfest24-demo-bucket"  # @param {type:"string"}
instance_name="devfest24-demo" # @param {type:"string"}
database_password = 'testing' # @param {type:"string"}
database_name = 'testing' # @param {type:"string"}
database_user = 'testing' # @param {type:"string"}

assert database_name, "⚠️ Please provide a database name"
assert database_user, "⚠️ Please provide a database user"
assert database_password, "⚠️ Please provide a database password"

# dont update variable below
!gcloud config set project {project_id} --quiet

cloudrun_services = !gcloud run services describe courses-api --region=us-central1 --format='value(status.url)'
api_base_url = cloudrun_services[0]

staging_bucket_uri = f"gs://{staging_bucket_name}"
# get the ip address of the cloudsql instance
ip_addresses = !gcloud sql instances describe {instance_name} --format="value(ipAddresses[0].ipAddress)"
database_host = ip_addresses[0]

gemini_embedding_model = "text-embedding-004"
gemini_llm_model = "gemini-1.5-pro-001"
embeddings_table_name = "course_content_embeddings"
chat_history_table_name = "chat_histories"

print(f"API Base URL: {api_base_url}")
print(f"Database Host: {database_host}")

### Initializing vertex ai

In [3]:
import vertexai
vertexai.init(project=project_id, location=region, staging_bucket=staging_bucket_uri)

### Agent Code

This section below define a few classes that we have tried from the previous notebook.

The main focus here should be `CourseAgent` where we setup all dependencies required by the agent

In [4]:
class CourseAPIClient:
  def __init__(self, url=api_base_url):
    self.url = url
    
  def list_courses(self):
      response = requests.get(f"{self.url}/courses")
      return response.json()

  def get_course(self, course_name):
      response = requests.get(f"{self.url}/courses/{course_name}")
      return response.json()

  def create_order(self, course, user_name, user_email):
      payload = {
          "course": course,
          "user_name": user_name,
          "user_email": user_email
      }
      response = requests.post(f"{self.url}/orders", json=payload)
      return response.json()

  def get_order(self, order_id):
      response = requests.get(f"{self.url}/orders/{order_id}")
      return response.json()

  def pay_order(self, order_id):
      response = requests.post(f"{self.url}/orders/{order_id}:pay")
      return response.json()

  def get_payment_page_url(self, order_id):
      return f"{self.url}/orders/{order_id}/payment"

class GetCourseInput(BaseModel):
    course: str = Field(description="name of the course. this is the unique identifier of the course. it typically contains the course title with dashes, all in lowercase.")

class GetOrderInput(BaseModel):
    order_number: str = Field(description="order number identifier. this is a unique identifier in uuid format.")

class CreateOrderInput(BaseModel):
    course: str = Field(description="name of the course. this is the unique identifier of the course. it typically contains the course title with dashes, all in lowercase.")
    user_name: str = Field(description="name of the user who is purchasing the course .")
    user_email: str = Field(description="email of the user who is purchasing the course.")

class CourseAgent(reasoning_engines.Queryable):
    def __init__(
        self,
        model: str,
        project: str,
        region: str,
        instance: str,
        database: str,
        table: str,
        user: Optional[str] = None,
        password: Optional[str] = None,
        
    ):
        self.model_name = model
        self.project = project
        self.region = region
        self.instance = instance
        self.database = database
        self.table = table
        self.user = user
        self.password = password                
        self.store = {}   
        self.agent = None
        self.retriever = None 
        self.engine = None    

    def __getstate__(self):
        """Custom method for pickling the object."""
        state = self.__dict__.copy()
        # Remove the unpicklable entries
        del state['agent']
        del state['retriever']
        del state['engine']
        return state

    def __setstate__(self, state):
        """Custom method for unpickling the object."""
        self.__dict__.update(state)
        self.agent = None
        self.retriever = None
        self.engine = None
        # Note: set_up() will need to be called after unpickling


    def list_courses(self) -> List[str]:
        """List all available courses sold on the platform."""        
        client = CourseAPIClient()
        return client.list_courses()

    def create_order(self, course: str, user_name: str, user_email: str) -> str:
        """Create order for a course. This function can be used to create an order for a course. When this function returns successfully, it will return payment url to user to make payment. """        
        client = CourseAPIClient()
        print(f"Creating order for course: {course}, user_name: {user_name}, user_email: {user_email}")
        
        res = client.create_order(course, user_name, user_email)
        print(res)
        order_id = res["order_id"]
        payment_url = f"{api_base_url}/orders/{order_id}/payment"
        return f"Order number {order_id} created successfully. Payment URL: {payment_url}"

    def get_course(self, course: str) -> str:        
        """Get course details by course name. course name is the unique identifier of the course. it typically contains the course title with dashes.
        This function can be used to get course details such as course price, etc."""        
        client = CourseAPIClient()
        return client.get_course(course)
    
    def get_order(self, order_number: str) -> str:
        """Get order by using order number. This function can be used to get order details such as payment status to check whether the order has been paid or not. If user already paid the course, say thanks"""        
        client = CourseAPIClient()
        return client.get_order(order_number)

    def search_course_content(self, query: str) -> str:
        """Explain about software security course materials."""
        result = str(self.retriever.invoke(query))
        return result        

    def get_session_history(self, session_id: str) -> BaseChatMessageHistory:
        return PostgresChatMessageHistory.create_sync(
            self.engine,
            table_name=chat_history_table_name,
            session_id=session_id,
        )

    def set_up(self):
        """All unpickle-able logic should go here.
        In general, add any logic that requires a network or database
        connection.
        """

        #  we must initialize the Vertex AI client to use the right project and location
        # otherwise, when the agent is initialized in reasoning engine, it will use reasoning engine project
        vertexai.init(project=self.project, location=self.region)

        # Initialize the vector store
        self.engine = PostgresEngine.from_instance(
            self.project,
            self.region,
            self.instance,
            self.database,
            user=self.user,
            password=self.password,
            quota_project=self.project,
        )

        embeddings_service = VertexAIEmbeddings(model_name=gemini_embedding_model, 
                                                project=self.project)
        
        vector_store = PostgresVectorStore.create_sync(
            self.engine,
            table_name=self.table,
            embedding_service=embeddings_service,
        )
        self.retriever = vector_store.as_retriever(search_kwargs={"k": 10})
                
        search_course_content = StructuredTool.from_function(
            func=self.search_course_content,
            name="search_course_content",
            description="Explain about software security course materials.",            
        )

        list_courses = StructuredTool.from_function(
            func=self.list_courses,
            name="list_courses",
            description="List all available courses sold on the platform.",
        )

        get_course = StructuredTool.from_function(
            func=self.get_course,
            name="get-course-tool",
            description="""Get course details by course name. course name is the unique identifier of the course. it typically contains the course title with dashes.
        This function can be used to get course details such as course price, etc.""",
            args_schema=GetCourseInput,
        )

        create_order = StructuredTool.from_function(
            func=self.create_order,
            name="create-order-tool",
            description="""Create order for a course. This function can be used to create an order for a course. When this function returns successfully, it will return payment url to user to make payment.""",
            args_schema=CreateOrderInput,
        )

        get_order = StructuredTool.from_function(
            func=self.get_order,
            name="get-order-tool",
            description="""Get order by using order number. This function can be used to get order details such as payment status to check whether the order has been paid or not. If user already paid the course, say thanks""",
            args_schema=GetOrderInput,
        )
        
        tools = [search_course_content, list_courses, get_course, create_order, get_order]

        # Initialize the LLM and prompt
        prompt = {
            "chat_history": lambda x: x["history"],
            "input": lambda x: x["input"],
            "agent_scratchpad": (
                lambda x: format_to_openai_function_messages(x["intermediate_steps"])
            ),
        } | ChatPromptTemplate(
            messages = [
                SystemMessagePromptTemplate.from_template("""
                You are a bot assistant that sells online course about software security. You only use information provided from datastore or tools. You can provide the information that is relevant to the user's question or the summary of the content. If they ask about the content, you can give them more detail about the content. If the user seems interested, you may suggest the user to enroll in the course. 
                """),
                MessagesPlaceholder(variable_name="chat_history", optional=True),
                HumanMessagePromptTemplate.from_template("Use tools to answer this questions: {input}"),
                MessagesPlaceholder(variable_name="agent_scratchpad"),
            ]
        )

        safety_settings = {
            HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        }

        ## Model parameters
        model_kwargs = {
            "temperature": 0.5,
            "safety_settings": safety_settings,
        }

        self.agent = reasoning_engines.LangchainAgent(
            model=self.model_name,
            tools=tools,
            prompt=prompt,    
            chat_history=self.get_session_history,
            agent_executor_kwargs={
                "return_intermediate_steps": True,
            },
            model_kwargs=model_kwargs,
            enable_tracing=True,
        )
        print("agent is configured")


    def query(self, input: str, session_id: str) -> str:
        """Query the application.

        Args:
            input: The user query.
            session_id: The user's session id.

        Returns:
            The LLM response dictionary.
        """        

        response = self.agent.query(
            input=input,
            config={"configurable": {"session_id": session_id}},
        )
        return response

### Instantiating the agent

In [ ]:
agent = CourseAgent(
    model=gemini_llm_model,
    project=project_id,
    region=region,
    instance=instance_name,
    database=database_name,
    table=embeddings_table_name,    
    user=database_user,
    password=database_password,           
)
agent.set_up()

### Testing agent (again) locally

In [ ]:
import uuid

# Generate a UUID for the session ID
session_id = str(uuid.uuid4())
print(f"Generated session ID: {session_id}")

In [ ]:
res = agent.query(
    input="Can you please share what are taught on this course?", 
    session_id=session_id)

display(Markdown(res["output"]))

In [ ]:
response = agent.query(
    input="Does it teach about how to design a forgot password system securely?", 
    session_id=session_id)  
display(Markdown(response["output"]))

In [ ]:
response = agent.query(
    input="how much it costs?", 
    session_id=session_id)  
display(Markdown(response["output"]))

In [ ]:
response = agent.query(
    input="I want to order. name is mulyono, email is fufufafa@gmail.com", 
    session_id=session_id)  
display(Markdown(response["output"]))

In [ ]:
response = agent.query(
    input="I have made payment, can you please check my order status?", 
    session_id=session_id)  
display(Markdown(response["output"]))

### Deploying the Agent on Vertex AI

Before deploying the agent, we are going to grant excessive permissions for the agent first. Ideally, you should only grant the permissions that are required by the agent. But just to make sure that the agent is able to run without any issues, we are going to grant all the permissions for now. :p

In [ ]:
# Retrieve the project number associated with your project ID
from googleapiclient import discovery
service = discovery.build("cloudresourcemanager", "v1")
request = service.projects().get(projectId=project_id)
response = request.execute()
project_number = response["projectNumber"]
project_number

In [ ]:
!gcloud projects add-iam-policy-binding {project_id} \
    --member=serviceAccount:service-{project_number}@gcp-sa-aiplatform-re.iam.gserviceaccount.com \
    --role="roles/serviceusage.serviceUsageConsumer"

!gcloud projects add-iam-policy-binding {project_id} \
    --member=serviceAccount:service-{project_number}@gcp-sa-aiplatform-re.iam.gserviceaccount.com \
    --role="roles/discoveryengine.editor"

!gcloud projects add-iam-policy-binding {project_id} \
    --member=serviceAccount:service-{project_number}@gcp-sa-aiplatform-re.iam.gserviceaccount.com \
    --role="roles/cloudsql.admin"

!gcloud projects add-iam-policy-binding {project_id} \
    --member=serviceAccount:service-{project_number}@gcp-sa-aiplatform-re.iam.gserviceaccount.com \
    --role="roles/aiplatform.admin"

!gcloud projects add-iam-policy-binding {project_id} \
    --member=serviceAccount:service-{project_number}@gcp-sa-aiplatform-re.iam.gserviceaccount.com \
    --role="roles/aiplatform.user"

#### Deploying to reasoning engine

Deploying is as simple as calling `create()` method. We will provide the agent here and some dependencies required to run the agent.

In [ ]:
remote_agent = reasoning_engines.ReasoningEngine.create(
    agent,
    requirements=[
        "google-cloud-aiplatform",
        "google-cloud-aiplatform[langchain]",
        "google-cloud-aiplatform[reasoningengine]",
        "langchain",
        "langchain_core",
        "langchain_community",
        "langchain-google-vertexai==2.0.8",
        "cloudpickle",
        "pydantic==2.9.2",
        "langchain-google-community",
        "google-cloud-discoveryengine",
        "nest-asyncio",
        "asyncio==3.4.3",
        "asyncpg==0.29.0",
        "cloud-sql-python-connector[asyncpg]",
        "langchain-google-cloud-sql-pg",
        "numpy",
        "pandas",
        "pgvector",
        "psycopg2-binary",
        "google-cloud-trace"
    ],    
    display_name="course-agent",
    sys_version="3.11",
    
)
remote_agent

# remote_agent = reasoning_engines.ReasoningEngine.create(
#     agent,
#     requirements=[
#         "google-cloud-aiplatform==1.73.0",
#         "google-cloud-aiplatform[langchain]",
#         "google-cloud-aiplatform[reasoningengine]",
#         "langchain==0.3.9",
#         "langchain_core==0.3.21",
#         "langchain_community==0.3.9",
#         "langchain-google-vertexai==2.0.8",
#         "cloudpickle==3.1.0",
#         "pydantic==2.9.2",
#         "langchain-google-community==2.0.3",
#         "google-cloud-discoveryengine==0.13.4",
#         "nest-asyncio==1.6.0",
#         "asyncio==3.4.3",
#         "asyncpg==0.29.0",
#         "cloud-sql-python-connector[asyncpg]==1.13.0",
#         "langchain-google-cloud-sql-pg==0.11.1",
#         "numpy",
#         "pandas",
#         "pgvector",
#         "psycopg2-binary",
#         "google-cloud-trace==1.13.5"
#     ],    
#     display_name="course-agent",
#     sys_version="3.11",
    
# )
# remote_agent

### Testing Remote Agent

In [ ]:
from vertexai.preview import reasoning_engines

reasoning_engines.ReasoningEngine.list()
# remote_agent = reasoning_engines.ReasoningEngine("projects/908311267620/locations/us-central1/reasoningEngines/7151100481752793088")


In [ ]:
import uuid

# Generate a UUID for the session ID
session_id = str(uuid.uuid4())
print(f"Generated session ID: {session_id}")

In [ ]:
# Testing the remote agent
response = remote_agent.query(
  input="Can you please share what are being taught on this course",
  session_id=session_id,
)
display(Markdown(response["output"]))

In [ ]:
# Testing the remote agent
response = remote_agent.query(
  input="Does it teach about how to design a forgot password system securely?",
  session_id=session_id,
)
display(Markdown(response["output"]))

In [ ]:
# Testing the remote agent
response = remote_agent.query(
  input="How much this course costs?",
  session_id=session_id,
)
display(Markdown(response["output"]))

In [ ]:
# Testing the remote agent
response = remote_agent.query(
  input="Yes. I want to enroll",
  session_id=session_id,
)
display(Markdown(response["output"]))

In [ ]:
# Testing the remote agent
response = remote_agent.query(
  input="Name is Mulyono and email is mulyono@gmail.com",
  session_id=session_id,
)
display(Markdown(response["output"]))

In [ ]:
# Testing the remote agent
response = remote_agent.query(
  input="I have made the payment. Can you please check?",
  session_id=session_id,
)
display(Markdown(response["output"]))